# 推薦幫手初始化
OpenAI api 要計費，手下留情

In [1]:
import pandas as pd
import pinecone
from ast import literal_eval
from typing import List, Iterator
import numpy as np
import openai 
openai.api_key=""
EMBEDDING_MODEL = "text-embedding-ada-002"
api_key=""
pinecone.init(api_key=api_key,environment="northamerica-northeast1-gcp")
index_name = 'ntut-courses'
index = pinecone.Index(index_name=index_name)
pinecone.list_indexes()

df = pd.read_csv("./datasets/112-1embedding.csv")
df['description_embedding'] = df.description_embedding.apply(literal_eval)
df['id'] = df['id'].apply(str)
titles_mapped = dict(zip(df.id,df.title))
content_mapped = dict(zip(df.id,df.description))
def query_course(query, namespace, top_k=5):
    '''Queries an article using its title in the specified
     namespace and prints results.'''

    # Create vector embeddings based on the title column
    embedded_query = openai.Embedding.create(
                                            input=query,
                                            model=EMBEDDING_MODEL,
                                            )["data"][0]['embedding']

    # Query namespace passed as parameter using title vector
    query_result = index.query(embedded_query, 
                                      namespace=namespace, 
                                      top_k=top_k)

    # Print query results 
    print(f'\nMost similar results to {query} in "{namespace}" namespace:\n')
    if not query_result.matches:
        print('no query result')
    
    matches = query_result.matches
    ids = [res.id for res in matches]
    scores = [res.score for res in matches]
    df = pd.DataFrame({'id':ids, 
                       'score':scores,
                       'title': [titles_mapped[_id] for _id in ids],
                       'content': [content_mapped[_id] for _id in ids],
                       })
    
    counter = 0
    for k,v in df.iterrows():
        counter += 1
        print(f'{v.id} {v.title} (score = {v.score})')
    
    print('\n')

    return df
def generate_html(courses, scores, user):
    base_html = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>北科好課推推推</title>
        <style>
            .container {{
                display: flex;
                justify-content: center;
                align-items: center;
                flex-wrap: wrap;
            }}
            
            .navbar {{
                text-align: center;
                background-color: #f0f0f0;
                padding: 10px;
                font-size: 18px;
                font-weight: bold;
            }}
            
            .iframe-container {{
                width: 500px;
                margin: 10px;
            }}
            
            .iframe-container iframe {{
                height: 600px;
                width: 100%;
                border: none;
            }}
        </style>
    </head>
    <body>
        <div class="navbar">
            北科好課推薦
        </div>
        <div><h2>根據您的敘述: "{user}"，為您推薦5門課程</h2></div>
        <div class="container">
            {0}
        </div>
    </body>
    </html>
    """

    course_html = """
        <div class="iframe-container">
            <iframe frameborder="0" src="https://ntut-course.gnehs.net/course/112/1/{course}?mode=iframe"></iframe>
            <p>推薦指數{score}%</p>
        </div>
    """

    courses_html = [course_html.format(course=c, score=s) for c, s in zip(courses, scores)]
    return base_html.format("".join(courses_html), user=user)

c:\Users\Yupooooo\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# Recommand
修改 case 的字串再執行，可以得到網頁輸出5門課程推薦


先將使用者輸入Embedding後，再與向量資料庫進行比對

In [4]:

case1 = "給我幾個課程是和區塊鏈(Blockchain)有關的"
case2 = "我想學習關於其他國家的文化"
case3 = "我想要找跟炒股相關的課程"

case=case1

query_output = query_course(case,'content')
query_output["id"].to_list()
score=query_output["score"].to_list()
for i in range(len(score)):
    score[i] = int(score[i]*10000)/100
id=query_output["id"].to_list()
html_content = generate_html(id, score,case)
with open('main.html', 'w') as f:
    f.write(html_content)
import webbrowser
webbrowser.open("main.html")



Most similar results to 給我幾個課程是和區塊鏈(Blockchain)有關的 in "content" namespace:

319650 區塊鏈應用設計 (score = 0.876140296)
322389 金融科技 (score = 0.851414263)
321908 數位金融 (score = 0.837188423)
319681 可程式控制與實習 (score = 0.833771408)
320360 基礎程式設計 (score = 0.83328)




True